In [ ]:
%tensorflow_version 2.x
!pip install -q sklearn
try:
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.12.0-dev20221023


In [ ]:
# getting data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-10-24 00:54:32--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.6’

train-data.tsv.6    100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2022-10-24 00:54:32 (25.9 MB/s) - ‘train-data.tsv.6’ saved [358233/358233]

--2022-10-24 00:54:32--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.6’

valid-data.tsv.6    100%[========

In [ ]:
df_train = pd.read_table(train_file_path, delimiter= '\t', header = 0, names = ['ham/spam', 'message'])
df_train.head()


,ham/spam,message
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [ ]:
df_test = pd.read_table(test_file_path, delimiter= '\t', header = 0, names = ['ham/spam', 'message'])
df_test.head()

,ham/spam,message
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [ ]:
len(df_train)

4178

In [ ]:
len(df_test)

1391

In [ ]:
train_label = df_train.pop('ham/spam')
train_label = train_label.replace(['ham','spam'],[0,1])
ds_train = tf.data.Dataset.from_tensor_slices(
    (df_train, train_label)
)

In [ ]:
train_label.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    1
8    0
9    0
Name: ham/spam, dtype: int64

In [ ]:
test_label = df_test.pop('ham/spam')
test_label = test_label.replace(['ham','spam'],[0,1])

ds_test = tf.data.Dataset.from_tensor_slices(
    (df_test, test_label)
)

In [ ]:
def input_funct(ds, label, shuffle = True, batch_size=64):
  if shuffle:
    ds = ds.shuffle(len(ds))
  ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
  return ds

ds_train = input_funct(ds_train, train_label)
ds_test = input_funct(ds_test, test_label,shuffle = False)

In [ ]:
len(df_train)

4178

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
vectored_text = tf.keras.layers.TextVectorization(
    max_tokens=len(df_train)+100,
    output_mode='int',
    output_sequence_length=1500)

text_only_ds_train = ds_train.map(lambda text, labels: text)
vectored_text.adapt(text_only_ds_train)
#vectored_text.get_vocabulary()

In [ ]:
from keras.layers import Flatten, Embedding, Bidirectional, Dense

In [ ]:
model = tf.keras.Sequential()


In [ ]:
model.add(Flatten())

In [ ]:
model.add(vectored_text)

In [ ]:
model.add(Embedding(
        input_dim=len(vectored_text.get_vocabulary()),
        output_dim=64,
        mask_zero=True))

In [ ]:
model.add(Bidirectional(tf.keras.layers.LSTM(64)))

In [ ]:
model.add(Dense(64, activation='relu'))

In [ ]:
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model_check = model.fit(
    ds_train,
    validation_data=ds_test,
    validation_steps=30,
    epochs=6,
    verbose = 1, 
)

Epoch 1/6
66/66 [==============================] - ETA: 0s - loss: 0.2535 - accuracy: 0.9292

66/66 [==============================] - 257s 3s/step - loss: 0.2535 - accuracy: 0.9292 - val_loss: 0.2774 - val_accuracy: 0.9547
Epoch 2/6
66/66 [==============================] - 198s 3s/step - loss: 0.0951 - accuracy: 0.9813
Epoch 3/6
66/66 [==============================] - 202s 3s/step - loss: 0.0394 - accuracy: 0.9921
Epoch 4/6
66/66 [==============================] - 198s 3s/step - loss: 0.0266 - accuracy: 0.9931
Epoch 5/6
66/66 [==============================] - 201s 3s/step - loss: 0.0189 - accuracy: 0.9957
Epoch 6/6
66/66 [==============================] - 199s 3s/step - loss: 0.0151 - accuracy: 0.9976


In [ ]:
testing_char = model.evaluate(ds_test, verbose=0)

In [ ]:
str1 = "Loss "
str2 = "Accuracy "
print(str1,testing_char[0])
print(str2,testing_char[1])

Loss  0.03950926288962364
Accuracy  0.9863407611846924


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_model = model.predict([pred_text])
  pred = pred_model[0][0]
  if pred>0.5:
    return [pred, "spam"]
  else:
    return [pred, "ham"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 3s 3s/step
[-0.06118207, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too",
                   "have a package delivery, verify your email/icloud/android ID, or that a family member needs help urgently",
                   "hi pal, are you free today for a dinner ?"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham", "spam", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 186ms/step
You passed the challenge. Great job!
